# Find Scoary Genes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# read annotation file
gold_anno = pd.read_pickle('/home/hermuba/data0118/goldstandard/ec_rmplasmid_node_anno_df')

def read_scoary(abx):
    fname = '~/data0118/scoary/{}_12_09_2019_0534.results.csv'.format(abx)
    df = pd.read_csv(fname, header = 0, index_col = 0)
    return(df)

df = read_scoary('meropenem')
df = df.loc[(df['Odds_ratio']>8) & (df['Bonferroni_p']<0.01)]

In [2]:
df.shape

(663, 17)

# Basic network statistics

In [3]:
nodes = gold_anno.loc[gold_anno['cluster'].isin(df.index)]

In [4]:
nodes['GO'].count()/nodes.shape[0]

0.2790346907993967

In [5]:
nodes['strict_best_ARO'].count()/nodes.shape[0]

0.012066365007541479

In [6]:
nodes['resfam'].count()/nodes.shape[0]

0.03469079939668175

In [7]:
nodes['hypo_nr'].sum()/nodes.shape[0]

0.5641025641025641

In [ ]:
# run Gene Ontology Enrichment Analysis

In [9]:
def extract_anno(df):
    ''' return percentage to GO annotations'''
    clus_name = df.index.tolist()
    anno_subset = gold_anno.loc[gold_anno['cluster'].isin(clus_name)]
    
    return anno_subset
def anno_stat(anno_subset):
    go_perc = anno_subset['GO'].count()/anno_subset.shape[0]
    hypo_perc = anno_subset['hypo_nr'].count()/anno_subset.shape[0]
    card_perc = anno_subset['is_card'].sum()/anno_subset.shape[0]
    resfam_perc = anno_subset['resfam'].count()/anno_subset.shape[0]
    
    return [go_perc, hypo_perc, card_perc, resfam_perc, anno_subset.shape[0]]
def extract_go_term(series):
    ''' extract all go term'''
    # remove nan
    series.dropna(inplace = True)
    
    all_go_terms = set()
    for sid in series.index:
        go = series[sid]
        #go = go.split(',')
        #go = [g.replace('{','').replace('}','').replace('\'','').replace('\"','') for g in go]
        #series[sid] = go
        all_go_terms = all_go_terms.union(go)
        #print(all_go_terms)
    # make into table
    df = pd.DataFrame(index = series.index, columns = all_go_terms)
    for sid in series.index:
        df.loc[sid, list(series[sid])] = True
    df.fillna(False, inplace =True)
    
    return(df) 
onto_root = '/home/hermuba/data0118/ontologies/'
from goatools.obo_parser import GODag
obodag = GODag(onto_root + "go-basic.obo")

from goatools.associations import read_associations
ns2assoc = read_associations('/home/hermuba/data0118/ontologies/ec_rmplasmid.id2go', anno_type='id2gos', namespace = 'BP', no_top=True)
from goatools.go_enrichment import GOEnrichmentStudy 
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

goea = GOEnrichmentStudy(gold_anno.index.tolist(), ns2assoc, obodag, propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh'])
results_nt = goea.run_study(nodes.index.tolist())
# filter to significant 
goea_results_sig = [r for r in results_nt if r.p_fdr_bh < 0.05]
    
# plot it
outdir = '/home/hermuba/data0118/fig/'
plot_results(outdir+"meropenem_{NS}.png", goea_results_sig)
    
# write text to file
go_id = [g.GO for g in goea_results_sig]
goresult_df = pd.DataFrame([[obodag[gid].name, obodag[gid].namespace, gid] for gid in go_id], columns = ['Name', 'Namespace', 'ID'])

    

/home/hermuba/data0118/ontologies/go-basic.obo: fmt(1.2) rel(2019-07-01) 47,413 GO Terms
HMS:0:00:00.100394  26,164 annotations READ: /home/hermuba/data0118/ontologies/ec_rmplasmid.id2go 
**ERROR IdToGosReader(..., godag=None).get_id2gos: GODAG is None. IGNORING namespace(BP)

10044 IDs in all associations
Load GOEA Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 32% 10,044 of 31,621 population items found in association

Run GOEA Gene Ontology Analysis: current study set of 663 IDs ... 28%    185 of    663 study items found in association
100%    663 of    663 study items found in population(31621)
Calculating 2,067 uncorrected p-values using fisher_scipy_stats
   2,067 GO terms are associated with 10,044 of 31,621 population items
     164 GO terms are associated with    185 of    663 study items
  METHOD fdr_bh:
       3 GO terms found significant (< 0.05=alpha) (  3 enriched +   0 purified): statsmodels fdr_bh
      10 study items 

In [10]:
goresult_df

,Name,Namespace,ID
0,aromatic compound catabolic process,biological_process,GO:0019439
1,C4-dicarboxylate transport,biological_process,GO:0015740
2,C4-dicarboxylate transmembrane transporter act...,molecular_function,GO:0015556


In [56]:
[n for n in nodes.dropna(subset = ['GO']).index if 'GO:0019439' in nodes.loc[n,'GO']]

['562.22936.con.0074_6|562.22936',
 'FLWH01000010_53|562.12962',
 'JMUY01000005_146|1438670.3',
 'JMUY01000005_147|1438670.3',
 'JMUY01000005_148|1438670.3']

# Build subet and Look at individual community

In [11]:
# read baysean integrated combined network
import networkx as nx
with open('/home/hermuba/data0118/network1122/combined_rm_plasmid_baye', 'rb') as f:
    combined = nx.read_edgelist(f, nodetype = str, comments = '#', delimiter = ',',  data=(('combined_lls',float),))
mero_subnet = combined.subgraph(nodes.index)

In [12]:
'JMUY01000018_13|1438670.3' in mero_subnet.nodes()

True

# COG

In [16]:
from Genome.pangenome_annotate.gold_anno_downstream import *
popular_cogs = count_cog(nodes)

In [20]:
popular_cogs.iloc[-10:]

,no_genes,name
COG0270,3,Site-specific DNA-cytosine methylase
COG1961,3,Site-specific DNA recombinase related to the D...
COG4644,3,"Transposase and inactivated derivatives, TnpA ..."
COG1192,3,Cellulose biosynthesis protein BcsQ
COG1974,3,SOS-response transcriptional repressor LexA (R...
COG4245,3,"Uncharacterized conserved protein YegL, contai..."
COG3505,4,"Type IV secretory pathway, VirD4 component, Tr..."
COG2704,4,Anaerobic C4-dicarboxylate transporter
COG4974,5,Site-specific recombinase XerD
COG2310,5,Stress response protein SCP2


In [23]:
domain = count_domain(nodes)

In [27]:
domain.loc[domain['ENTRY_TYPE'] == 'Domain'].iloc[:20]

,ENTRY_TYPE,ENTRY_NAME,no_genes
ENTRY_AC,,,
IPR002104,Domain,"Integrase, catalytic domain",7
IPR003325,Domain,TerD domain,6
IPR025668,Domain,Transposase DDE domain,5
IPR002035,Domain,"von Willebrand factor, type A",5
IPR001387,Domain,Cro/C1-type helix-turn-helix domain,5
IPR020846,Domain,Major facilitator superfamily domain,5
IPR001584,Domain,"Integrase, catalytic core",5
IPR014001,Domain,"Helicase superfamily 1/2, ATP-binding domain",5
IPR022038,Domain,"Ig-like domain, bacterial type",4


In [28]:
notnull=nodes.loc[nodes['nr'].notnull()]
adhesion = notnull.loc[notnull['nr'].str.contains('VWA')]

In [29]:
# VWA
from itertools import combinations
d = []
for g1,g2 in combinations(adhesion.index,2):
    d.append(nx.algorithms.shortest_paths.generic.shortest_path_length(mero_subnet, g1, g2))

In [30]:
gold_anno.loc[list(mero_subnet.neighbors(adhesion.index[3])), 'nr'].values

array(['WP_000762697.1 MULTISPECIES: VWA domain-containing protein [Enterobacteriaceae]',
       'WP_031942303.1 DNA-binding protein [Salmonella enterica]',
       'WP_000081059.1 MULTISPECIES: protein phosphatase 2C domain-containing protein [Enterobacterales]',
       'WP_001253656.1 MULTISPECIES: VWA domain-containing protein [Enterobacterales]',
       'WP_001053340.1 MULTISPECIES: tellurium resistance protein TerF [Enterobacterales]',
       'WP_012006601.1 VWA domain-containing protein [Enterobacter cloacae]'],
      dtype=object)

In [31]:
gna = gold_anno.loc[gold_anno['nr'].notnull()]
gna.loc[gna['nr'].str.contains('RNA')]

,cog_ID,cog_category,cluster,count,core,loose_best_ARO,loose_ARO,strict_best_ARO,strict_ARO,is_card,resfam,aclame_title,is_aclame,nr,hypo_nr,drug_target,is_drug_target,GO,pathway,domain
562.10576.con.0004_47|562.10576,NaN,NaN,Cluster 17604,6.0,False,NaN,NaN,NaN,NaN,False,NaN,NaN,False,WP_001054916.1 DNA-directed RNA polymerase sig...,NaN,NaN,False,"{GO:0016987, GO:0003677, GO:0003700, GO:000635...",NaN,"{IPR013249, IPR013324, IPR036388}"
562.10576.con.0013_74|562.10576,COG0124,J,Cluster 4741,1579.0,True,NaN,NaN,NaN,NaN,False,NaN,protein:proph:175647 Length: 426 # NCBI annota...,True,WP_000037894.1 MULTISPECIES: histidine--tRNA l...,NaN,NaN,False,"{GO:0005737, GO:0006427, GO:0005524, GO:0004821}",{KEGG: 00970+6.1.1.21},"{IPR033656, IPR041715, IPR015807, IPR036621, I..."
562.22444.con.0044_8|562.22444,COG1595,K,Cluster 18594,3.0,False,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:155991 Length: 173 # NCBI anno...,True,WP_001609597.1 DNA-directed RNA polymerase sig...,NaN,NaN,False,"{GO:0016987, GO:0003677, GO:0003700, GO:000635...",NaN,"{IPR007627, IPR013325, IPR000838}"
562.22449.con.0002_140|562.22449,COG0564,J,Cluster 13563,1579.0,True,NaN,NaN,NaN,NaN,False,NaN,NaN,False,WP_000525177.1 bifunctional tRNA pseudouridine...,NaN,NaN,False,"{GO:0009982, GO:0001522, GO:0009451, GO:0003723}",NaN,"{IPR006145, IPR020103, IPR006225, IPR006224}"
562.22453.con.0021_61|562.22453,COG3270,J,Cluster 3408,1579.0,True,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:135552 Length: 404 # NCBI anno...,True,WP_097734804.1 16S rRNA (cytosine(1407)-C(5))-...,NaN,NaN,False,"{GO:0006364, GO:0008649, GO:0008757, GO:000372...","{Reactome: R-HSA-6790901, Reactome: R-HSA-8869...","{IPR031341, IPR029063, IPR001678, IPR027391, I..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JMUY01000009_57|1438670.3,COG0030,J,Cluster 11144,1579.0,True,Erm(30),"ARO:3001265, ARO:3000522, ARO:3000599, ARO:300...",NaN,NaN,False,"{Erm23S_rRNA_methyltrans, Erm38, ErmA, ErmB, E...",protein:plasmid:147589 Length: 244 # NCBI anno...,True,WP_001065365.1 16S rRNA (adenine(1518)-N(6)/ad...,NaN,NaN,False,"{GO:0000154, GO:0006364, GO:0008649, GO:0000179}","{Reactome: R-HSA-2151201, Reactome: R-HSA-6793...","{IPR001737, IPR029063, IPR020596, IPR020598, I..."
JMUY01000009_63|1438670.3,COG0553,L,Cluster 562,1579.0,True,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:16436 Length: 979 # NCBI annot...,True,WP_001117001.1 MULTISPECIES: RNA polymerase-as...,NaN,NaN,False,"{GO:0006355, GO:0005524, GO:0016817}",NaN,"{IPR000330, IPR022737, IPR027417, IPR014001, I..."
JMVL01000002_288|1438683.3,NaN,NaN,Cluster 28870,1.0,False,NaN,NaN,NaN,NaN,False,NaN,NaN,False,WP_001524198.1 tRNA 2-selenouridine(34) syntha...,NaN,NaN,False,NaN,NaN,NaN
JMVP01000002_87|1438687.3,COG3344,L,Cluster 2402,1.0,False,NaN,NaN,NaN,NaN,False,NaN,protein:plasmid:18224 Length: 462 # NCBI annot...,True,WP_032195564.1 MULTISPECIES: RNA-directed DNA ...,NaN,NaN,False,"{GO:0006278, GO:0003723, GO:0003964}",NaN,"{IPR000123, IPR000477}"


In [33]:
nx.write_gexf(mero_subnet, '/home/hermuba/data0118/network1122/mero_subnet.xml', encoding='utf-8', prettyprint=True, version='1.2draft')

In [35]:
for e in mero_subnet.edges:
    mero_subnet.edges[e]['lls'] = mero_subnet.edges[e]['combined_lls']
    del mero_subnet.edges[e]['combined_lls']
# gml no underscore


NetworkXError: 'combined_lls' is not a valid key

In [46]:
nx.write_gml(mero_subnet, '/home/hermuba/data0118/network1122/mero_subnet.gml')